In [ ]:
!pip install -U sentence-transformers
!pip install plotly==4.14.1
!pip install torch

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
import sys
sys.path.append('/content/drive/My Drive/data/icns_project')

In [4]:
import torch
import pandas as pd
from sentence_transformers import SentenceTransformer, util
from pathlib import Path

from sklearn.metrics.pairwise import euclidean_distances, cosine_distances
from scipy.spatial.distance import euclidean, pdist, squareform
from sklearn import manifold          #use this for MDS computation

#visualization libs
import plotly.express as px
import plotly.graph_objects as go
import matplotlib.pyplot as plt
% matplotlib inline

In [5]:
MODEL_PATH = Path('drive') / 'My Drive' / 'data' / 'icns_project' / 'paraphrase-distilroberta-base-v1'
DATA_PATH = Path('drive') / 'My Drive' / 'data' / 'icns_project'

In [6]:
pd.set_option('max_colwidth', 800)

In [7]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print('using device: ', torch.cuda.get_device_name(device), flush=True)

using device:  Tesla T4


In [8]:
model = SentenceTransformer(str(MODEL_PATH))

In [9]:
df = pd.read_csv(DATA_PATH / 'jokes_stupid_wocka_relevant.csv', encoding='utf-8')

In [10]:
news_df = pd.read_csv(DATA_PATH / 'BBC_news_adjusted.csv', encoding='utf-8')

In [11]:
news_df.shape

(1490, 2)

In [12]:
news_df = news_df.rename({'Text': 'text', 'Category': 'category'}, axis=1)

In [13]:
news_df['category'].value_counts()

sport            346
business         336
politics         274
entertainment    273
tech             261
Name: category, dtype: int64

In [14]:
df['category'].value_counts()

At Work            264
News / Politics    258
Sports             166
Tech               126
Political          109
Money               68
Computers           55
Business            40
Office Jokes        17
Name: category, dtype: int64

In [15]:
df['category'] = df['category'].replace({'At Work': 'business',
                'News / Politics': 'politics',
                'Sports': 'sport',
                'Tech': 'tech',
                'Political': 'politics',
                'Money': 'business',
                'Computers': 'tech',
                'Business': 'business',
                'Office Jokes': 'business'})

In [16]:
df['category'].value_counts()

business    389
politics    367
tech        181
sport       166
Name: category, dtype: int64

In [17]:
df.shape

(1103, 5)

In [18]:
df = df.dropna(subset=['text'])

In [19]:
df.shape

(1103, 5)

In [20]:
news_df.shape

(1490, 2)

In [21]:
news_df = news_df.dropna(subset=['text'])

In [22]:
news_df.shape

(1490, 2)

In [23]:
df = df.assign(source='joke')
news_df = news_df.assign(source='news')

In [24]:
df.head(2)

,text,category,source,score,length
0,"A brunette, a blonde, and a redhead all worked in the same office with the same female boss. Every day, they noticed their boss left work early. One day, the girls decided that when the boss left, they'd leave right behind her. After all, she never called in or came back to the office when she left early, so how was she to know? The next day, they all three left the office right after the boss left. The brunette was thrilled to be home early. She did a little gardening and went to bed early. The redhead was elated to be able to get in a quick workout at the health club before meeting her dinner date. The blonde was happy, happy, happy to be home, but when she got to the bedroom she heard a muffled noise from inside. Slowly, quietly, she cracked open the door and was mortified to se...",business,joke,3.73,260
1,"Bill and Hillary Clinton went out to dinner and when the waiter came to take their order, he asked Bill how he wanted his steak, she replied, ""medium."" Then the waiter said, ""how about your vegetable?"" Bill replied, ""Oh, she can order for herself.""",politics,joke,3.50,59


In [25]:
news_df.head(2)

,text,category,source
0,worldcom ex-boss launches defence lawyers defending former worldcom chief bernie ebbers against a battery of fraud charges have called a company whistleblower as their first witness. cynthia cooper worldcom s ex-head of internal accounting alerted directors to irregular accounting practices at the us telecoms giant in 2002.,business,news
1,german business confidence slides german business confidence fell in february knocking hopes of a speedy recovery in europe s largest economy. munich-based research institute ifo said that its confidence index fell to 95.5 in february from 97.5 in january its first decline in three months.,business,news


In [26]:
df = df[['text', 'category', 'source']]

In [27]:
combined = pd.concat([df, news_df], ignore_index=True)

In [28]:
combined.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2593 entries, 0 to 2592
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   text      2593 non-null   object
 1   category  2593 non-null   object
 2   source    2593 non-null   object
dtypes: object(3)
memory usage: 60.9+ KB


In [29]:
combined['source'].value_counts()

news    1490
joke    1103
Name: source, dtype: int64

In [33]:
print("Max Sequence Length:", model.max_seq_length)

Max Sequence Length: 128


In [34]:
# tests with preprocessing and model max seq length

In [35]:
combined['text'] = combined.text.str.lower()

In [36]:
combined['text'] = combined.text.str.replace('[!?"]', '')

In [37]:
tmp_jokes = combined[(combined['source'] == 'joke') & (combined['category'] == 'sport')] #.sample(5)

In [38]:
tmp_jokes

,text,category,source
6,"two men are approaching each other on a sidewalk. both are dragging their right foot as they walk. as they meet, one man looks at the other knowingly, points to his foot and says, vietnam, 1969. the other points his thumb behind him and says, dog crap, 20 feet back.",sport,joke
14,"a man went to visit his 90 year old grandfather in a very secluded rural area of the state he lived in.after spending the night, his grandfather prepared breakfast for him consisting of eggs and bacon. he noticed a film like substance on his plate and he questioned his grandfather, are these plates cleanhis grandfather replied, those plates are as clean as cold water can get them, so go on and finish your meal.that afternoon, while eating the hamburgers his grandfather made for lunch, he noticed tiny specks around the edge of his plate and a substance that looked like dried egg yokes, so he ask again, are you sure these plates are cleanwithout looking up from his hamburger, the grandfather says, i told you before; those dishes are as clean as cold water can get them. now don't ask me a...",sport,joke
17,"a man is walking down the street when he sees a sign in the window of a travel agency that says cruises - $100. he goes into the agency and hands the guy $100. the travel agent then whacks him over the head with a baseball bat and throws him in the river.another man is walking down the street a half hour later, sees the sign and pays the guy $100. the travel agent then whacks him with the baseball bat and throws him in the river.sometime later, the two men are floating down the river together and the first man asks, do you think they'll serve any food on this cruisethe second man says, i don't think so. they didn't do it last year.",sport,joke
18,"how was your golf game, dear asked jack's wife tracy.well, i was hitting pretty well, but my eyesight's gotten so bad icouldn't see where the ball went.but you're seventy-five years old, jack admonished his wife,why don't you take my brother scott alongbut he's eighty-five and doesn't even play golf anymore,protested jack.but he's got perfect eyesight. he could watch your ball,tracy pointed out.the next day jack teed off with scott looking on. jack swung,and the ball disappeared down the middle of the fairway.do you see it asked jack.yup, scott answered.well, where is it yelled jack, peering off into the distance.i forgot.",sport,joke
23,"it is the olympic men's figure skating. out comes the russian competitor, he skates around to some classical music in a slightly dull costume, performs some excellent leaps but without any great artistic feel for the music. the judges' scores read: britain 5.8: russia 5.9: united states 5.5: ireland 6.0 next comes the american competitor in a sparkling stars and stripes costume, skating to some rock and roll music. he gets the crowd clapping, but is not technically as good as the russian. he slightly misses landing a triple salchow and loses the center during a spin. but, artistically, it is a more satisfying performance. the judges' scores read: britain 5.8: russia 5.5: united states 5.9: ireland 6.0 finally out comes the irish competitor wearing a tatty old donkey jacket, wit...",sport,joke
...,...,...,...
1068,"top ten signs you're not watching a real baseball team from late show with david letterman; monday, february 20, 1995 you recognize batter as the kid who sold you a hot dog a couple minutes earlier. every time a player slides into second, he busts his hip. they keep shouting do over when umpire yells, strike 3 batter looks at him as if the dude's speaking french. try as they might, they just can't scratch themselves like professionals. first base: siskel. second base: ebert. game stops when some lady in a house near the stadium shouts dinner time players constantly adjusting each other's cups. you overheard the coach yelling, run, forrest, run they play like the mets.",sport,joke
1087,"two mates calles mik and mak went to the australia v

In [39]:
tmp_news = combined[(combined['source'] == 'news') & (combined['category'] == 'sport')] #.sample(5)

In [40]:
tmp_news

,text,category,source
1109,wales silent on grand slam talk rhys williams says wales are still not thinking of winning the grand slam despite a third six nations win. that s the last thing on our minds at the moment said williams a second- half replacement in saturday s 24-18 win over france in paris. we all realise how difficult a task it is to go up to scotland and beat them.,sport,news
1117,ireland 21-19 argentina an injury-time dropped goal by ronan o gara stole victory for ireland from underneath the noses of argentina at lansdowne road on saturday. o gara kicked all of ireland s points with two dropped goals and five penalties to give the home side a 100% record in their autumn internationals.,sport,news
1118,wenger signs new deal arsenal manager arsene wenger has signed a new contract to stay at the club until may 2008. wenger has ended speculation about his future by agreeing a long-term contract that takes him beyond the opening of arsenal s new stadium in two years.,sport,news
1120,hantuchova in dubai last eight daniela hantuchova moved into the quarter-finals of the dubai open after beating elene likhotseva of russia 7-5 6-4 and now faces serena williams. australian open champion williams survived an early scare to beat russia s elena bovina 1-6 6-1 6-4.,sport,news
1121,melzer shocks agassi in san jose second seed andre agassi suffered a comprehensive defeat by jurgen melzer in the quarter-finals of the sap open. agassi was often bamboozled by the austrian s drop shots in san jose losing 6-3 6-1.,sport,news
...,...,...,...
2570,charvis set to lose fitness bid flanker colin charvis is unlikely to play any part in wales final two games of the six nations. charvis has missed all three of wales victories with an ankle injury and his recovery has been slower than expected.,sport,news
2571,preview: ireland v england (sun) lansdowne road dublin sunday 27 february 1500 gmt bbc1 radio 4 lw and this website ireland are going for their first grand slam since 1948 after two opening wins and england represent their sternest test of the championship so far. england were sloppy and leaderless in the defeats against wales and france and another loss would be unthinkable.,sport,news
2574,ferrero eyes return to top form former world number one juan carlos ferrero insists he can get back to his best despite a tough start to 2005. the 2003 french open champion has slipped to 64 in the world after a year of illness and injuries in 2004 but is confident that his form will return.,sport,news
2576,dallaglio eyeing lions tour place former england captain lawrence dallaglio still harbours hopes of a place on the british and irish lions tour to new zealand. lions coach sir clive woodward has made it clear he will pick his squad to tour next summer based on form shown in the six nations championship.,sport,news


In [41]:
tmp = pd.concat([tmp_jokes, tmp_news], ignore_index=True)

In [43]:
#Change max sequence length of the model
model.max_seq_length = 512

In [44]:
model.max_seq_length

512

In [45]:
embs = model.encode(tmp.text.to_list(), show_progress_bar=True)

In [46]:
embs_df = pd.DataFrame(embs)

In [47]:
embs_df

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,728,729,730,731,732,733,734,735,736,737,738,739,740,741,742,743,744,745,746,747,748,749,750,751,752,753,754,755,756,757,758,759,760,761,762,763,764,765,766,767
0,0.124801,-0.007502,-0.008061,-0.005189,-0.357371,0.523320,-0.022404,-0.018851,0.123623,-0.041797,-0.042617,-0.261050,0.052231,-0.694182,0.028858,0.258708,0.603574,-0.114871,-0.185400,0.050888,0.065316,-0.452855,-0.034172,0.136222,0.061083,-0.004258,-0.085004,0.248412,-0.239900,0.113040,0.202940,-0.103075,0.234498,0.076903,0.099550,-0.067800,-0.089362,-0.207321,0.372120,0.002016,...,0.025600,0.498334,0.046252,0.164974,0.427859,0.269142,0.190710,-0.443522,-0.034914,0.211937,0.122616,-0.010568,-0.095072,-0.137024,-0.169129,-0.603798,-0.181073,-0.000143,0.103652,-0.028710,-0.209232,0.236913,0.099972,0.017351,-0.304927,-0.087690,0.169769,-0.172516,0.360386,0.164597,0.275574,-0.195388,-0.030630,0.156924,-0.378563,-0.053482,-0.125059,0.014793,-0.162782,-0.146230
1,-0.440565,-0.388667,0.120421,0.146600,-0.107786,0.482386,0.148933,-0.182777,-0.134912,0.145673,0.426800,-0.299215,0.246872,-0.341138,0.108592,0.133444,0.386603,-0.189184,-0.012221,0.101361,0.074140,-0.601051,0.317717,-0.503686,-0.301281,0.323193,-0.014825,0.301043,-0.266526,-0.053858,0.041876,-0.033287,-0.214337,0.091766,0.127729,0.129340,-0.112310,0.063693,0.165862,-0.353857,...,0.230978,-0.012204,0.056761,-0.128374,0.354661,0.058618,0.280752,0.436821,-0.216821,0.296208,-0.124122,-0.195952,-0.337281,-0.170968,0.088699,-0.179560,-0.095593,0.284447,-0.057676,-0.048946,-0.017151,-0.192518,0.035687,-0.201521,-0.204375,-0.094253,-0.155097,-0.045881,0.478202,-0.040711,-0.079108,-0.148914,0.172370,0.081551,0.159946,0.046242,-0.133694,-0.009447,-0.004923,0.225714
2,-0.108351,0.102725,0.099066,0.327774,-0.253966,0.276951,-0.058987,0.206664,0.012456,-0.143818,0.027132,-0.151205,0.192661,-0.430688,-0.229284,0.133078,0.322186,0.019446,-0.178153,-0.191965,-0.120992,-0.317578,0.576165,0.397094,0.051321,-0.037175,-0.036071,0.304217,-0.123776,0.192836,-0.245184,-0.021134,0.230507,0.067431,-0.519733,0.263417,-0.172570,-0.181054,0.060265,-0.045414,...,0.190236,0.195125,0.206841,0.077960,0.063285,-0.057929,-0.051617,0.025408,-0.029202,-0.108091,-0.297006,-0.184294,-0.048830,0.172152,0.076167,-0.157908,-0.404812,0.040477,-0.262699,-0.343525,0.269857,-0.237136,-0.347432,-0.041823,-0.062414,-0.199282,-0.174428,-0.360430,-0.303079,0.218031,0.124127,-0.042666,-0.151603,-0.047960,-0.077064,0.088717,0.316170,-0.266200,-0.135109,0.066520
3,-0.172755,-0.024257,0.287079,0.354603,0.036258,-0.236290,0.123815,0.368460,-0.078089,0.515048,0.027041,-0.140275,-0.009657,0.172755,-0.216334,-0.359980,0.156651,0.121532,0.375928,0.422618,-0.026608,-0.471621,0.631019,-0.025203,-0.001070,0.268262,-0.007469,0.194734,-0.049154,-0.551102,-0.140005,0.038793,-0.379330,-0.016869,0.847081,-0.357709,-0.168592,-0.032210,0.308380,0.048992,...,0.041419,0.165998,-0.054800,0.019107,0.116942,-0.488891,0.291450,0.327640,0.128516,-0.383425,0.226028,-0.401835,-0.299775,-0.368451,-0.003701,-0.258262,-0.198315,-0.196581,0.035381,-0.166843,-0.284522,-0.681257,-0.216721,-0.062588,0.120291,-0.060421,0.132651,0.302597,0.305927,-0.148068,-0.164079,-0.078832,-0.367222,0.300387,-0.355860,0.083487,0.132766,0.118764,0.137366,-0.441846
4,0.309381,0.213125,0.318156,0.078030,-0.084411,0.074532,0.065993,0.227206,0.105908,-0.226032,-0.118648,-0.098966,-0.057197,0.041466,0.512825,0.054719,0.321883,-0.260506,-0.020826,-0.078433,-0.080534,-0.240825,0.284356,0.222655,-0.357929,0.054241,-0.017163,-0.106364,-0.077471,-0.049230,-0.138492,0.116872,-0.317832,0.018145,-0.195839,-0.046130,0.338885,0.223355,0.062173,-0.187660,...,0.247024,0.171568,-0.058484,0.096137,0.210807,0.359044,0.198724,-0.148763,0.261447,0.040798,0.081029,-0.169521,-0.309695,-0.433509,0.545614,-0.337076,0.202597,-0.183630,0.019440,-0.400993,0.230056,0.029754,-0.604228,-0.096231,-0.087210,0.379543,0.648215,-0.36384

In [48]:
embs_df.transpose().describe()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,472,473,474,475,476,477,478,479,480,481,482,483,484,485,486,487,488,489,490,491,492,493,494,495,496,497,498,499,500,501,502,503,504,505,506,507,508,509,510,511
count,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,...,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000
mean,0.008383,0.007813,0.008609,0.010324,0.011249,0.011247,0.012340,0.010199,0.010649,0.015192,0.013112,0.010884,0.011032,0.013306,0.010773,0.010857,0.007912,0.011249,0.009327,0.014063,0.009165,0.010134,0.010724,0.013132,0.007196,0.010975,0.011154,0.008783,0.010885,0.012326,0.010982,0.011375,0.012669,0.011260,0.012851,0.012012,0.009859,0.007775,0.009168,0.012856,...,0.019997,0.021188,0.015279,0.013249,0.017938,0.012024,0.014459,0.018590,0.016335,0.014673,0.019311,0.017823,0.013994,0.020489,0.012504,0.019640,0.016341,0.017480,0.011842,0.017007,0.017657,0.019150,0.014841,0.023010,0.016807,0.017835,0.016461,0.018141,0.015468,0.015401,0.019617,0.016269,0.015152,0.020780,0.018314,0.015465,0.012598,0.016476,0.022709,0.017338
std,0.232771,0.248371,0.233986,0.277749,0.262441,0.262000,0.263876,0.248191,0.263226,0.269756,0.287510,0.310758,0.273775,0.284643,0.253349,0.266960,0.265993,0.264407,0.268207,0.281420,0.245942,0.251048,0.259060,0.276340,0.274549,0.270554,0.274915,0.255753,0.260993,0.266715,0.283019,0.295548,0.274309,0.262529,0.280776,0.331685,0.226129,0.261728,0.272893,0.261782,...,0.237058,0.259547,0.256140,0.291800,0.268534,0.264883,0.264002,0.261186,0.308803,0.271706,0.250001,0.306463,0.313663,0.260659,0.307223,0.295719,0.277377,0.248139,0.293337,0.310214,0.311650,0.256593,0.307579,0.288128,0.277176,0.273036,0.285904,0.304133,0.234853,0.296453,0.330398,0.262534,0.253156,0.275252,0.266536,0.277110,0.291007,0.275589,0.299514,0.272820
min,-0.838565,-0.892408,-0.690200,-0.883562,-0.716802,-0.803782,-0.922871,-1.097413,-0.926055,-1.101339,-0.894965,-1.183661,-0.995697,-0.896388,-0.853154,-0.902948,-0.850177,-0.713111,-1.029004,-0.993810,-0.776286,-0.902375,-0.835059,-0.948397,-0.803132,-0.959821,-0.846076,-0.981429,-1.094031,-1.128427,-1.043060,-1.018773,-1.127726,-0.737719,-1.022741,-1.192447,-0.810840,-0.858715,-0.938284,-0.778956,...,-0.799493,-0.901638,-0.752174,-1.026862,-0.814830,-1.183698,-0.928720,-0.891640,-1.155786,-1.142387,-0.860511,-1.045708,-1.171755,-0.768229,-0.897481,-0.926903,-0.908037,-0.812749,-0.877755,-1.005630,-0.945595,-0.797889,-1.004676,-0.759678,-1.015326,-0.824135,-1.098868,-1.192093,-0.849988,-0.838658,-1.215055,-0.840273,-0.822131,-1.061406,-1.334195,-0.984684,-1.010945,-0.810901,-0.893988,-0.901392
25%,-0.133815,-0.143101,-0.148938,-0.164643,-0.163799,-0.162756,-0.156714,-0.149981,-0.155065,-0.157303,-0.165535,-0.182648,-0.154853,-0.179658,-0.149028,-0.156646,-0.180336,-0.161895,-0.162278,-0.163554,-0.143528,-0.150479,-0.147539,-0.155006,-0.170764,-0.160897,-0.149981,-0.152164,-0.155873,-0.157014,-0.179413,-0.192924,-0.169080,-0.157247,-0.172595,-0.218068,-0.137139,-0.173856,-0.173182,-0.158397,...,-0.123131,-0.157030,-0.147196,-0.171683,-0.160745,-0.149859,-0.153828,-0.152332,-0.164804,-0.144821,-0.143736,-0.188257,-0.186671,-0.158489,-0.189275,-0.192167,-0

In [49]:
dists = cosine_distances(embs)

In [50]:
pd.DataFrame(dists)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,472,473,474,475,476,477,478,479,480,481,482,483,484,485,486,487,488,489,490,491,492,493,494,495,496,497,498,499,500,501,502,503,504,505,506,507,508,509,510,511
0,0.000000,0.804008,0.611524,0.838090,0.887655,0.734218,0.861425,0.892721,0.772563,0.999438,0.913523,0.839673,0.866924,0.825452,0.833746,0.884100,0.794782,0.896601,0.966519,0.945126,0.856114,0.933680,0.859546,0.855024,0.728006,0.926368,0.795188,0.789157,0.807942,0.974090,0.896360,0.883247,0.949232,0.895341,0.882195,0.977561,0.875331,0.688975,0.865159,0.876261,...,1.031101,1.042575,0.997013,1.027095,1.014575,0.946267,0.864189,0.861165,0.862951,0.990310,0.963783,1.074816,1.019518,0.994144,0.906049,1.080042,0.954287,0.974923,0.971294,1.119280,1.017251,0.966174,0.976404,0.946353,0.958478,0.980651,0.999502,0.960962,0.957858,0.996294,1.033428,0.991139,1.033241,1.015346,1.065712,1.056708,0.864915,1.063638,0.985495,1.046800
1,0.804008,0.000000,0.705750,0.702487,0.871664,0.879395,0.825131,0.689362,0.684797,0.918848,0.927515,0.801486,0.632830,0.804877,0.757593,0.711485,0.719815,0.775116,0.749641,0.863183,0.722957,0.844545,0.716303,0.817512,0.717082,0.906866,0.828081,0.726282,0.706683,0.617707,0.865801,1.022371,0.866825,0.888074,0.841587,0.954046,0.990288,0.739778,0.754672,1.023376,...,0.902390,0.973715,1.059232,0.878857,0.874972,0.897511,0.868826,0.825871,0.910299,0.989915,0.930460,0.958925,0.872596,1.043495,0.842037,0.975234,0.961161,0.952678,0.854519,1.006568,0.978177,0.981524,0.905619,0.973790,0.929908,0.937164,0.974279,0.945981,0.893058,0.949921,0.993972,0.852436,0.915044,0.893539,1.029099,0.966783,0.709911,0.923666,0.943887,0.870621
2,0.611524,0.705750,0.000000,0.854874,0.821748,0.743286,0.874793,0.820606,0.734203,0.986632,0.887754,0.839805,0.759378,0.780478,0.727906,0.862578,0.729899,0.779063,0.945049,0.967628,0.804758,0.980227,0.805347,0.850630,0.597880,0.898007,0.866702,0.712278,0.802920,0.888006,0.719544,0.853115,0.858888,0.809344,0.773476,0.872643,0.947146,0.615057,0.863523,1.081201,...,0.983065,0.954720,1.009807,0.917222,0.881296,0.972018,0.852194,0.897535,0.786831,1.063934,0.996300,0.918212,0.824414,0.946697,0.747951,0.948998,0.934056,0.874045,0.916521,1.051670,1.058907,0.996250,0.984431,1.008515,1.007003,0.963119,0.975802,1.093715,0.844179,0.982408,1.015083,0.960557,0.993176,1.016215,0.999719,0.951576,0.817206,1.077836,1.018535,0.986985
3,0.838090,0.702487,0.854874,0.000000,0.938336,0.834913,0.752231,0.532144,0.470034,0.796702,0.835286,0.670121,0.666155,0.857423,0.669896,0.736693,0.507556,0.750770,0.441199,0.848857,0.722967,0.729343,0.761054,0.759082,0.761628,0.764127,0.519943,0.501762,0.475712,0.724413,0.791198,0.858146,0.758242,0.957851,0.804960,0.966481,0.746157,0.722782,0.656732,0.858173,...,0.820190,0.882238,0.887064,0.850762,0.856749,0.783776,0.738372,0.796734,0.778928,0.820466,0.861335,0.891230,0.906901,0.937895,0.755003,0.879718,0.927549,0.930069,0.834542,0.819137,0.933172,0.827760,0.963099,0.795931,0.848982,0.840689,0.986087,0.832652,0.852038,0.846785,0.955916,0.903102,0.913754,0.911584,0.974292,0.804234,0.807362,0.876867,0.890554,0.903949
4,0.887655,0.871664,0.821748,0.938336,0.000000,0.609533,0.484572,0.786578,0.846929,0.859225,0.775213,0.875360,0.849999,0.712810,0.761626,0.828350,0.894125,0.807915,0.845512,0.653676,0.786551,0.896701,0.883713,0.846502,0.655267,0.870480,0.940360,0.840596,0.831832,0.979372,0.836044,0.845236,0.507673,0.010949,0.836810,0.662378,0.693068,0.643807,0.786616,0.878280,...,1.016072,0.824252,0.907082,0.765730,1.049024,0.759292,0.834032,0.822257,0.840201,0.793574,0.972149,0.742850,0.729106,0.994235,0.798808,0.703592,0.926507,0.821045,0.788185,0.692156,0.815022,0.971237,0.809258,0.891048,0.862404,0.993086,0.879272,0.879105,0.977453,0.690468,0.792409,0.932174,0.907707,0.923183,0.847516,0.806224,0.726590,0.815923,0.991971,0.927439
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...

In [51]:
mds = manifold.MDS(n_components=2, dissimilarity="precomputed", random_state=60, max_iter=1000)
results = mds.fit(dists)

In [52]:
results

MDS(dissimilarity='precomputed', eps=0.001, max_iter=1000, metric=True,
    n_components=2, n_init=4, n_jobs=None, random_state=60, verbose=0)

In [53]:
coords = results.embedding_

In [54]:
coords

array([[ 0.19516389, -0.80269268],
       [ 0.4745114 , -0.57505371],
       [ 0.63576998, -0.44124774],
       ...,
       [-0.32863639,  0.59158803],
       [ 0.02017577,  0.62558399],
       [ 0.22786369,  0.49439486]])

In [55]:
tmp['x'] = pd.Series(coords[:, 0])
tmp['y'] = pd.Series(coords[:, 1])

In [56]:
label_counts = tmp['source'].value_counts()

In [57]:
label_counts

news    346
joke    166
Name: source, dtype: int64

In [58]:
mask = tmp.source == 'news'

In [59]:
tmp[mask]

,text,category,source,x,y
166,wales silent on grand slam talk rhys williams says wales are still not thinking of winning the grand slam despite a third six nations win. that s the last thing on our minds at the moment said williams a second- half replacement in saturday s 24-18 win over france in paris. we all realise how difficult a task it is to go up to scotland and beat them.,sport,news,-0.245072,-0.117579
167,ireland 21-19 argentina an injury-time dropped goal by ronan o gara stole victory for ireland from underneath the noses of argentina at lansdowne road on saturday. o gara kicked all of ireland s points with two dropped goals and five penalties to give the home side a 100% record in their autumn internationals.,sport,news,-0.221561,-0.254971
168,wenger signs new deal arsenal manager arsene wenger has signed a new contract to stay at the club until may 2008. wenger has ended speculation about his future by agreeing a long-term contract that takes him beyond the opening of arsenal s new stadium in two years.,sport,news,0.500514,0.544391
169,hantuchova in dubai last eight daniela hantuchova moved into the quarter-finals of the dubai open after beating elene likhotseva of russia 7-5 6-4 and now faces serena williams. australian open champion williams survived an early scare to beat russia s elena bovina 1-6 6-1 6-4.,sport,news,-0.512856,0.127257
170,melzer shocks agassi in san jose second seed andre agassi suffered a comprehensive defeat by jurgen melzer in the quarter-finals of the sap open. agassi was often bamboozled by the austrian s drop shots in san jose losing 6-3 6-1.,sport,news,-0.575804,-0.230858
...,...,...,...,...,...
507,charvis set to lose fitness bid flanker colin charvis is unlikely to play any part in wales final two games of the six nations. charvis has missed all three of wales victories with an ankle injury and his recovery has been slower than expected.,sport,news,-0.244339,0.356218
508,preview: ireland v england (sun) lansdowne road dublin sunday 27 february 1500 gmt bbc1 radio 4 lw and this website ireland are going for their first grand slam since 1948 after two opening wins and england represent their sternest test of the championship so far. england were sloppy and leaderless in the defeats against wales and france and another loss would be unthinkable.,sport,news,-0.124969,-0.217596
509,ferrero eyes return to top form former world number one juan carlos ferrero insists he can get back to his best despite a tough start to 2005. the 2003 french open champion has slipped to 64 in the world after a year of illness and injuries in 2004 but is confident that his form will return.,sport,news,-0.328636,0.591588
510,dallaglio eyeing lions tour place former england captain lawrence dallaglio still harbours hopes of a place on the british and irish lions tour to new zealand. lions coach sir clive woodward has made it clear he will pick his squad to tour next summer based on form shown in the six nations championship.,sport,news,0.020176,0.625584


In [60]:
tmp['x'][mask]

166   -0.245072
167   -0.221561
168    0.500514
169   -0.512856
170   -0.575804
         ...   
507   -0.244339
508   -0.124969
509   -0.328636
510    0.020176
511    0.227864
Name: x, Length: 346, dtype: float64

In [61]:
colorscale = ['#00CC96', '#AB63FA', '#FFA15A', 
                '#19D3F3', '#FF6692', '#B6E880', '#FF97FF', '#FECB52']

In [62]:
fig = go.Figure()
idx = 0
for cat, count in label_counts.iteritems():
  mask = tmp.source == cat
  fig.add_trace(go.Scatter(x=tmp['x'][mask], y=tmp['y'][mask],
                mode='markers', textposition="top center",
                marker=dict(
                    size=10,
                    color=colorscale[idx],
                    opacity=0.8,
                ), text=tmp['text'][mask], hoverinfo='text',
                name=cat))
  idx += 1      

fig.update_layout(template="plotly_dark")
fig.update_layout(title_text=f'Comparison of embeddings from sport category')
fig.show()
fig.write_html(str(DATA_PATH) + "news_jokes_sports.html")

In [63]:
!pip install chart_studio

In [66]:
import chart_studio.tools as tls
import chart_studio.plotly as py

In [68]:
py.plot(fig, filename='news_jokes_sports')

'https://plotly.com/~karmenkk/1/'